In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html',
         'https://www.janestreet.com/puzzles/archive/page2/index.html',
         'https://www.janestreet.com/puzzles/archive/page3/index.html',
         'https://www.janestreet.com/puzzles/archive/page4/index.html',
         'https://www.janestreet.com/puzzles/archive/page5/index.html',
         'https://www.janestreet.com/puzzles/archive/page6/index.html',
         'https://www.janestreet.com/puzzles/archive/page7/index.html',
         'https://www.janestreet.com/puzzles/archive/page8/index.html',
         'https://www.janestreet.com/puzzles/archive/page9/index.html',
         'https://www.janestreet.com/puzzles/archive/page10/index.html',
         'https://www.janestreet.com/puzzles/archive/page11/index.html',
         'https://www.janestreet.com/puzzles/archive/page12/index.html']  
links = ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [93]:
# Work out the scores with an offset. Count as attempted all those since the name appeared on a leaderboard.
# No adjustments for similar names etc
# Borrowed the offset idea from this (much more professionally done) tableau league table 
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet

def df_format(styler,caption=""):
    styler.format({'Score':'{:.2%}','Solved':'{:.0f}','Attempted':'{:.0f}'})
    styler.set_table_attributes("style='display:inline'")
    styler.hide(axis=0,names=True)
    styler.set_caption(caption)
    styler.set_table_styles([{'selector': 'td', 'props': [('text-align', 'center'),
                                                          ('font-size', '12px')]},
                             {'selector': '.col_heading', 'props': [('text-align', 'center'),
                                                                    ('width', '30px'),
                                                                    ('background-color', 'slategrey'),
                                                                    ('color', 'white'),
                                                                   ('font-size', '12px')]},
                             {'selector': '.row_heading', 'props': [('text-align', 'left'),
                                                                    ('background-color', 'slategrey'),
                                                                    ('color', 'white'),
                                                                    ('font-size', '12px')]},
                             {'selector': 'caption', 'props': [ ('font-size', '18px'),('font-weight','bold'),('text-align', 'left')]},
                            ])
    styler.apply(lambda x: ['background: lightgreen'if x.name == 'Gareth Owen'  else '' for i in x], axis=1)
    my_name =[{'selector': '.row_heading', 'props': 'color : lightgreen'}]
    styler.set_table_styles({'Gareth Owen': my_name},axis=1,overwrite=False)
                                                      
    return styler

outputs = []

for newcomer_offset in [2.5]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
    outputs += [df2.sort_values(['Score','Solved'],ascending=False)[:15].style.pipe(df_format,caption="With an offset of "+str(newcomer_offset))._repr_html_()]

display(HTML(("".join(outputs))))

,Solved,Attempts,Score
Evan Semet,26,26,91.23%
Karl Mahlburg,44,50,83.81%
Senthil Rajasekaran,49,61,77.17%
Gareth Owen,32,41,73.56%
Lazar Ilic,29,38,71.60%
Aaditya Raghavan,33,44,70.97%
Blaine Hill,6,6,70.59%
Arthur Bright,12,15,68.57%
Calvin Pozderac,52,74,67.97%
Sean Egan,47,68,66.67%


In [94]:
df2.sort_values('Solved',ascending=False)[:15].style.pipe(df_format,caption="Number of Attempts")

,Solved,Attempts,Score
Calvin Pozderac,52,74,67.97%
Senthil Rajasekaran,49,61,77.17%
Sean Egan,47,68,66.67%
Karl Mahlburg,44,50,83.81%
Keith Schneider,41,65,60.74%
Heidi Stockton,39,85,44.57%
Aaditya Raghavan,33,44,70.97%
Sébastien Geeraert,32,59,52.03%
Gareth Owen,32,41,73.56%
Michael DeLyser,30,63,45.80%


In [95]:
def df_format(styler,caption=""):
    styler.format({'Score':'{:.2%}','Solved':'{:.0f}','Attempted':'{:.0f}'})
    styler.set_table_attributes("style='display:inline'")
    styler.hide(axis=0,names=True)
    styler.set_caption(caption)
    styler.set_table_styles([{'selector': 'td', 'props': [('text-align', 'center'),
                                                          ('font-size', '9px')]},
                             {'selector': '.col_heading', 'props': [('text-align', 'center'),
                                                                    ('width', '20px'),
                                                                    ('background-color', 'slategrey'),
                                                                    ('color', 'white'),
                                                                   ('font-size', '9px')]},
                             {'selector': '.row_heading', 'props': [('text-align', 'left'),
                                                                    ('background-color', 'slategrey'),
                                                                    ('color', 'white'),
                                                                    ('font-size', '9px')]},
                             {'selector': 'caption', 'props': [ ('font-size', '14px'),('font-weight','bold'),('text-align', 'left')]},
                            ])
    styler.apply(lambda x: ['background: lightgreen'if x.name == 'Gareth Owen'  else '' for i in x], axis=1)
    my_name =[{'selector': '.row_heading', 'props': 'color : lightgreen'}]
    styler.set_table_styles({'Gareth Owen': my_name},axis=1,overwrite=False)
                                                      
    return styler

outputs = []

for newcomer_offset in [0,0.5,1,2,10,15,20]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
    outputs += [df2.sort_values(['Score','Solved'],ascending=False)[:15].style.pipe(df_format,caption="With an offset of "+str(newcomer_offset))._repr_html_()]

display(HTML(("".join(outputs))))

,Solved,Attempts,Score
Evan Semet,26,26,100.00%
Blaine Hill,6,6,100.00%
Jake Zimelman,3,3,100.00%
Jonathan Prystupa,3,3,100.00%
Matthew Cihlar,2,2,100.00%
.David R Robinson,1,1,100.00%
Bootes,1,1,100.00%
Bradley Louie Saito,1,1,100.00%
Cakery,1,1,100.00%
Cathan Gormley,1,1,100.00%


In [30]:
print(df2.sort_values('Score',ascending=False)[:15].to_markdown())

| Name                |   Solved |   Attempted |    Score |
|:--------------------|---------:|------------:|---------:|
| Karl Mahlburg       |       44 |          50 | 0.733333 |
| Evan Semet          |       26 |          26 | 0.722222 |
| Senthil Rajasekaran |       49 |          61 | 0.690141 |
| Gareth Owen         |       32 |          41 | 0.627451 |
| Calvin Pozderac     |       52 |          74 | 0.619048 |
| Aaditya Raghavan    |       33 |          44 | 0.611111 |
| Lazar Ilic          |       29 |          38 | 0.604167 |
| Sean Egan           |       47 |          68 | 0.602564 |
| Cubist              |       30 |          43 | 0.566038 |
| Keith Schneider     |       41 |          65 | 0.546667 |
| Arthur Bright       |       12 |          15 | 0.48     |
| Dan Taylor          |       18 |          28 | 0.473684 |
| Andrew Sultana      |       18 |          28 | 0.473684 |
| Stephen Emet        |       21 |          35 | 0.466667 |
| Sébastien Geeraert  |       32 |      

In [31]:
# How does my position change with Newcomer Offset
for newcomer_offset in range(41):

    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempted']
    df2['Score']=df2['Solved']/(df2['Attempted']+ newcomer_offset/2)
    df3 = df2.sort_values('Score',ascending=False).reset_index() 
    print(newcomer_offset/2,df3[df3['Name'] =='Gareth Owen'].index[0]+1)

0.0 29
0.5 11
1.0 6
1.5 5
2.0 5
2.5 4
3.0 4
3.5 4
4.0 4
4.5 4
5.0 4
5.5 4
6.0 4
6.5 4
7.0 4
7.5 4
8.0 4
8.5 4
9.0 4
9.5 4
10.0 4
10.5 4
11.0 4
11.5 4
12.0 5
12.5 5
13.0 5
13.5 5
14.0 5
14.5 5
15.0 5
15.5 5
16.0 5
16.5 5
17.0 6
17.5 6
18.0 6
18.5 6
19.0 6
19.5 6
20.0 6
